# Telemetry Report Comparison

This notebook demonstrates how to load multiple telemetry tuning reports,
merge them, and visualize objective improvements using the helper
`analyze_tuner_reports.py`.


## Setup

We'll reuse the CLI helpers that generated the latest `demo_tuner_report.csv`
under `docs/examples/analytics/data/tuner_reports/`.


In [ ]:
from pathlib import Path
import pandas as pd

try:
    import altair as alt
except ImportError:
    alt = None

DATA_DIR = Path('docs/examples/analytics/data/tuner_reports')
BASELINE = DATA_DIR / 'demo_tuner_report.csv'
EXPERIMENT = DATA_DIR / 'demo_tuner_report.csv'  # replace with new report as needed

## Load Reports
We can either call the helper script via `subprocess` or load the CSVs directly
for ad-hoc comparisons.


> CI executes with a lightweight telemetry sweep, so the required CSV files are generated automatically.


In [ ]:
if not BASELINE.exists() or not EXPERIMENT.exists():
    print('Telemetry reports not found. Skipping comparison; run the telemetry sweep as described in docs/howto/telemetry_tuning.rst.')
    merged = pd.DataFrame()
else:
    baseline = pd.read_csv(BASELINE)
    experiment = pd.read_csv(EXPERIMENT)
    baseline['label'] = 'baseline'
    experiment['label'] = 'experiment'
    merged = pd.concat([baseline, experiment])
    display(merged)

## Visualize Best Objectives
Plot the best objective per algorithm to see improvements/deltas.


In [ ]:
if alt is None:
    display('Altair not installed; install `altair` to render the chart.')
else:
    chart = (
        alt.Chart(merged)
        .mark_line(point=True)
        .encode(
            x='label:N',
            y='best_objective:Q',
            color='algorithm:N',
            column='scenario:N'
        )
    )
    chart

## Delta Table
Join baseline and experiment to compute deltas using pandas.


In [ ]:
if merged.empty:
    comparison = pd.DataFrame()
    print('No comparison data loaded.')
else:
    comparison = baseline.merge(
        experiment, on=['algorithm', 'scenario'], suffixes=('_baseline', '_experiment')
    )
    comparison['best_delta'] = (
        comparison['best_objective_experiment'] - comparison['best_objective_baseline']
    )
comparison

Use this notebook as a starting point for richer analytics (e.g., trendlines
across multiple nightly reports).